In [1]:
import pandas as pd
df= pd.read_csv('churn_prediction_data/churn_prediction_data/train.csv')

##alter the categorial data to numerical type 
df_dummies = pd.get_dummies(df, columns=['Geography','Gender'], dtype=float)

##alter some extreme number by feature scaling
minmax_col = ['CreditScore', 'Balance', 'EstimatedSalary']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_dummies[minmax_col] = scaler.fit_transform(df_dummies[minmax_col])

df_dummies["Age"]=pd.to_numeric(df_dummies["Age"],downcast="float")
df_dummies["Tenure"]=pd.to_numeric(df_dummies["Tenure"],downcast="float")
df_dummies["NumOfProducts"]=pd.to_numeric(df_dummies["NumOfProducts"],downcast="float")
df_dummies["IsActiveMember"]=pd.to_numeric(df_dummies["IsActiveMember"],downcast="float")

df_dummies

all_inputs = df_dummies[['CreditScore','Age','Tenure','Balance','NumOfProducts',
                 'IsActiveMember','EstimatedSalary','Geography_France','Geography_Germany','Geography_Spain',
                 'Gender_Female','Gender_Male']].values
all_classes = df_dummies['Exited'].values



from sklearn.ensemble import RandomForestRegressor
# create the random forest with your hyperparameters.
model = RandomForestRegressor(n_estimators=340)
# fit the model to start training.
model.fit(all_inputs,all_classes)
# get the importance of the resulting features.
importances = model.feature_importances_
# create a data frame for visualization.
final_df = pd.DataFrame({"Importances":importances})
final_df.set_index('Importances')
all_inputs=all_inputs * importances


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#只選得分最好的k＝11個features.
skb=SelectKBest(chi2,k=11)
all_inputs=skb.fit_transform(all_inputs,all_classes)
print('after',all_inputs.shape)


#after ajusting data, begin to use it in algorithm to predict the result
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score
#使用網格搜索確定要建立的基學習器個數
clf = GridSearchCV(RandomForestClassifier(max_features="log2"),param_grid=({"n_estimators":range(1,101,10)}),cv=10)
clf.fit(all_inputs,all_classes)
print(clf.best_params_)
#再使用網格搜索來確定決策樹的參數
clf2 = GridSearchCV(RandomForestClassifier(n_estimators=11),param_grid=({"max_depth":range(1,10)}))
clf2.fit(all_inputs,all_classes)
print(clf2.best_params_)
#根據最大層數3，最多棵樹11，建立最終的隨機森林來預測
rf = RandomForestClassifier(n_estimators=11,max_depth=10,max_features="log2")
rf.fit(all_inputs,all_classes)




#adjusting the testing data and then use in algorithm  
test_df= pd.read_csv('churn_prediction_data/churn_prediction_data/test.csv')
test_df_dummies = pd.get_dummies(test_df, columns=['Geography','Gender'], dtype=float)
test_df_dummies.head(5)
##alter some extreme number
minmax_col = ['CreditScore', 'Balance', 'EstimatedSalary']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
test_df_dummies[minmax_col] = scaler.fit_transform(test_df_dummies[minmax_col])

test_df_dummies["Age"]=pd.to_numeric(test_df_dummies["Age"],downcast="float")
test_df_dummies["Tenure"]=pd.to_numeric(test_df_dummies["Tenure"],downcast="float")
test_df_dummies["NumOfProducts"]=pd.to_numeric(test_df_dummies["NumOfProducts"],downcast="float")
test_df_dummies["IsActiveMember"]=pd.to_numeric(test_df_dummies["IsActiveMember"],downcast="float")


alltest_inputs = test_df_dummies[['CreditScore','Age','Tenure','Balance','NumOfProducts',
                 'IsActiveMember','EstimatedSalary','Geography_France','Geography_Germany','Geography_Spain',
                 'Gender_Female','Gender_Male']].values
alltest_inputs=alltest_inputs * importances
alltest_inputs = skb.transform(alltest_inputs)

y_pred = rf.predict(alltest_inputs) 
print(y_pred)

import csv
file = open("churn_prediction_data/churn_prediction_data/upload13.csv", "w", newline='')
writer = csv.writer(file)
writer.writerow(['', 'RowNumber', 'Exited'])
for w in range(len(y_pred)):
    writer.writerow([w,test_df['RowNumber'][w], y_pred[w]])
file.close()

after (8000, 11)
{'n_estimators': 31}
{'max_depth': 8}
[0 0 0 ... 0 0 0]


## [Sklearn-RandomForest隨機森林參數及實例](https://www.twblogs.net/a/5d3f3ebbbd9eee51fbf903a0)

## [#使用網格搜索_集成學習（Random Forest）——實踐](https://www.itread01.com/content/1521194172.html)